In [ ]:
# Generalized Hebbian Algorithm (GHA) for extracting Prinicipal Components (PCA)
# Import Required Libraries
import numpy
from matplotlib import pyplot as plt
from LoadData import load_images, load_labels
from copy import deepcopy

In [ ]:
# PCA using Sample Covariance Matrix
Images = load_images('train-images-idx3-ubyte.gz')
Labels_Train = load_labels('train-labels-idx1-ubyte.gz')
Mean = numpy.mean(Images, axis=0)
Images_Norm = Images - Mean
row, col = Images.shape
Sample_Cov_Matrix = numpy.matmul(Images_Norm.T, Images_Norm)/(row - 1)
Eigen_Values, Eigen_Vectors = numpy.linalg.eig(Sample_Cov_Matrix)
Sort_Index = numpy.argsort(Eigen_Values)[::-1]
U = Eigen_Vectors[:, Sort_Index]

In [ ]:
# PCA using GHA Extracting First 3 PCs
# Since we want the first three prinicpal components our neural network will have only three outputs. And after training 
# the weights converge to the three PCs. 

# Parameters
W_Initial_Guess = numpy.random.normal(0, 0.5, (3, 784))
W_Initial_Guess_Norm = W_Initial_Guess/numpy.linalg.norm(W_Initial_Guess, axis=1).reshape(3, 1)
eta = 1e-9
epoch = 1
max_epoch = 150

# Required Functions
def update_weights(lr, x, W, iterations):
    y = numpy.dot(W, x)
    LT = numpy.tril(numpy.matmul(y[:, numpy.newaxis], y[numpy.newaxis, :]))
    W = W + lr/iterations * ((y[:, numpy.newaxis] * x) - (numpy.matmul(LT, W)))
    return W
    
# Main
W_new = deepcopy(W_Initial_Guess_Norm)
while epoch <= max_epoch:
    for i in range(0, row):
        W_new = update_weights(eta, Images_Norm[i], W_new, epoch)
    print('Epoch: ', epoch, ' LR: ', eta)
    epoch += 1
W_final = deepcopy(W_new)
print('Optimal Weights Reached!!!')

In [ ]:
# Test
W_final.T[numpy.abs(W_final.T) < 1e-5] = 0
print('Initial Guess:')
print(W_Initial_Guess)
print(W_Initial_Guess_Norm)
print('Final Sol:')
print(W_final)
print(W_final/numpy.linalg.norm(W_final, axis=1).reshape(3, 1))
print('Check if GHA 1st and 2nd PCs are orthogonal: ')
print(numpy.dot(W_final[0], W_final[1]))
print('Check if GHA 2nd and 3rd PCs are orthogonal: ')
print(numpy.dot(W_final[1], W_final[2]))
print('Check if GHA 1st and 3rd PCs are orthogonal: ')
print(numpy.dot(W_final[0], W_final[2]))
print('SVD and GHA Sol:')
print(U[:, 0:3])
print(W_final.T)
print('Check if SVD and GHA sol are close (euclidean distance):')
print(numpy.linalg.norm(W_final.T[:, 0] - U[:, 0]))
print(numpy.linalg.norm(W_final.T[:, 1] - U[:, 1]))
print(numpy.linalg.norm(W_final.T[:, 2] - U[:, 2]))
print('Check if 1st PC found using GHA and SVD are close (if value is 1 then it is close): ')
print(numpy.dot(W_final.T[:, 0], U[:, 0]))
print('Check if 2nd PC found using GHA and SVD are close (if value is 1 then it is close): ')
print(numpy.dot(W_final.T[:, 1], U[:, 1]))
print('Check if 3rd PC found using GHA and SVD are close (if value is 1 then it is close): ')
print(numpy.dot(W_final.T[:, 2], U[:, 2]))